In [1]:
import numpy as np
import cv2 as cv

if __name__ == '__main__':
    W_in = 224
    H_in = 224
    
    # input이 동영상인지 영상(이미지)인지 여부
    isVideo = True

    prototxt = "colorization_deploy_v2.prototxt.txt"
    caffemodel = "colorization_release_v2.caffemodel"
    kernel = "pts_in_hull.npy"
    input = "test_video3.mp4"

    # Select desired model
    net = cv.dnn.readNetFromCaffe(prototxt, caffemodel)

    pts_in_hull = np.load(kernel) # load cluster centers

    # populate cluster centers as 1x1 convolution kernel
    pts_in_hull = pts_in_hull.transpose().reshape(2, 313, 1, 1)
    net.getLayer(net.getLayerId('class8_ab')).blobs = [pts_in_hull.astype(np.float32)]
    net.getLayer(net.getLayerId('conv8_313_rh')).blobs = [np.full([1, 313], 2.606, np.float32)]

    if input:
        if "jpg" in input or "png" in input or "bmp" in input or "jpeg" in input:
            isVideo = False  # input이 이미지(영상)일 때
        cap = cv.VideoCapture(input)
    else:
        cap = cv.VideoCapture(0)

    width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))

    imshowSize = (width, height)
    
    if isVideo:  # input이 동영상일 때
        # 초 당 프레임 수 & 딜레이 구하기
        fps = round(cap.get(cv.CAP_PROP_FPS))
        delay = round(1000 / fps)
        
        # print(imshowSize, fps, delay)
        
        # 4-문자 코드 지정
        fourcc = cv.VideoWriter_fourcc(*'mp4v')
        # 동영상 파일 정의
        out = cv.VideoWriter("colorization3.mp4", fourcc, fps, (width, height), True)

    while cv.waitKey(1) < 0:
        hasFrame, frame = cap.read()
        if not hasFrame:
            cv.waitKey()
            print("DONE !!!")
            break

        img_rgb = (frame[:,:,[2, 1, 0]] * 1.0 / 255).astype(np.float32)

        img_lab = cv.cvtColor(img_rgb, cv.COLOR_RGB2Lab)
        img_l = img_lab[:,:,0] # pull out L channel
        (H_orig,W_orig) = img_rgb.shape[:2] # original image size

        # resize image to network input size
        img_rs = cv.resize(img_rgb, (W_in, H_in)) # resize image to network input size
        img_lab_rs = cv.cvtColor(img_rs, cv.COLOR_RGB2Lab)
        img_l_rs = img_lab_rs[:,:,0]
        img_l_rs -= 50 # subtract 50 for mean-centering

        net.setInput(cv.dnn.blobFromImage(img_l_rs))
        ab_dec = net.forward()[0,:,:,:].transpose((1,2,0)) # this is our result

        (H_out,W_out) = ab_dec.shape[:2]
        ab_dec_us = cv.resize(ab_dec, (W_orig, H_orig))
        img_lab_out = np.concatenate((img_l[:,:,np.newaxis],ab_dec_us),axis=2) # concatenate with original image L
        img_bgr_out = np.clip(cv.cvtColor(img_lab_out, cv.COLOR_Lab2BGR), 0, 1)
        
        frame = cv.resize(frame, imshowSize)
        
        if isVideo:  # input이 동영상일 때
            img_min = np.min(img_bgr_out)
            img_max = np.max(img_bgr_out)
            normalized_array = np.clip((img_bgr_out - img_min) * 255. / (img_max - img_min), 0, 255).astype(np.uint8)
            colorized = cv.resize(normalized_array, imshowSize)
            # 컬러화된 프레임 저장
            out.write(colorized)
        
        if cv.waitKey(delay) == 27:  # esc 누를 시
            break
        
        # 화면에 출력
        cv.imshow('origin', frame)
        cv.imshow('gray', cv.cvtColor(frame, cv.COLOR_RGB2GRAY))
        cv.imshow('colorized', cv.resize(img_bgr_out, imshowSize))
        
cap.release()
out.release()
cv.destroyAllWindows()
cv.waitKey(1)

DONE !!!


-1